In [1]:
import pandas as pd
import urllib.request
from tokenizers import BertWordPieceTokenizer

### 데이터 ETL

In [2]:
urllib.request.urlretrieve(
    'https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt',
    filename='ratings.txt'
)

# 데이터를 데이터프레임으로 불러오기
ratings_df = pd.read_table('ratings.txt')

# 결측값 제거
ratings_df = ratings_df.dropna(how='any')

# 전처리 데이터 저장
with open('ratings_1.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(ratings_df['document']))

### 토크나이저 설정

In [3]:
tokenizer = BertWordPieceTokenizer(
    # 대문자를 소문자로 변환하여 대소문자를 구분하지 않음
    lowercase=False,
    # 악센트 제거
    strip_accents=False
)

In [4]:
# 학습 데이터
files = 'ratings_1.txt'

# 단어 집합의 크기
vocab_size = 30000

# 허용하는 변환 전 텍스트의 토큰 개수
limit_alphabet = 6000

# 설정 값 이상으로 등장한 글자 쌍을 처리합니다
min_frequency = 5

tokenizer.train(
    files=files,
    vocab_size=vocab_size,
    limit_alphabet=limit_alphabet,
    min_frequency=min_frequency
)

# 단어 집합을 vocab.txt로 저장
tokenizer.save_model('.')

# 단어 집합 불러오기
vocab_df = pd.read_fwf('vocab.txt', header=None)

vocab_df

,0
0,[PAD]
1,[UNK]
2,[CLS]
3,[SEP]
4,[MASK]
...,...
29995,맛도
29996,망하지
29997,망한다
29998,망가지


### 토큰화

In [5]:
encoded = tokenizer.encode('아 배고픈데 짜장면먹고싶다')

print('토큰화 결과: ', encoded.tokens)
print('정수 인코딩 결과: ', encoded.ids)
print('디코딩 결과: ', tokenizer.decode(encoded.ids))

토큰화 결과:  ['아', '배고', '##픈', '##데', '짜장면', '##먹고', '##싶다']
정수 인코딩 결과:  [2111, 20631, 3980, 3242, 24681, 7873, 7379]
디코딩 결과:  아 배고픈데 짜장면먹고싶다
